In [32]:
import numpy as np
import gurobipy as gp
import matplotlib.pyplot as plt
from gurobipy import GRB
import re
import datetime
import ast
from tqdm.notebook import tqdm

In [33]:
from tests.utils import *

## Master problem correctness

In [34]:
file_name = "test7_slack11"
n = 200
K = 5
seed = 5
l = 3
alpha = 1/2
beta = [1, 1, 1]
lower = 2
upper = n
M = 1e6
np.random.seed(seed)
labels = np.random.choice([0, 1, 2], p=[0.1, 0.2, 0.7], size=n)

X, Y, archetype = synthetic_data(n, K, seed)
_, _, q, clusters_init, _, t = initialize_clusters(
    X, K, l, alpha, beta, n, labels, M, lower, upper)

Optimizing Cluster Centers: 100%|██████████| 300/300 [00:00<00:00, 2084.49it/s, Status=SUCCESS]


In [37]:
model = gp.read("./tests/model_write/" + file_name + "_out1000.lp")
constrs_len = len(model.getConstrs())
print(constrs_len)
clusters = np.loadtxt("./tests/model_matrix/"+file_name+"_clusters.txt")
distances = np.loadtxt("./tests/model_matrix/"+file_name+"_distances.txt")
slacks = np.loadtxt("./tests/model_matrix/"+file_name+"_slacks.txt")
solutions = np.loadtxt("./tests/model_matrix/"+file_name+"_solutions.txt")
objectives = np.loadtxt("./tests/model_matrix/"+file_name+"_objectives.txt")

Read LP format model from file ./tests/model_write/test7_slack11_out1000.lp
Reading time = 0.64 seconds
: 204 rows, 10180 columns, 889443 nonzeros
204


In [38]:
clusters = np.concatenate((clusters,np.ones(clusters.shape[0]).reshape(-1,1)),axis=1)
A = [[model.getCoeff(constr, var) for var in model.getVars()] for constr in model.getConstrs()]
b = [constr.rhs for constr in model.getConstrs()]
c = [var.obj for var in model.getVars()]
induced_mat = np.concatenate((clusters[:K,:],np.eye(constrs_len-1,constrs_len),clusters[K:,:]),axis=0).T
A = np.array(A)

Compare the linear system of constraints from the last iteration of Gurobi with the cluster representations

In [39]:
np.allclose(induced_mat[:,:A.shape[1]],A)

True

In [40]:
dist_gurobi = np.array(c[:K] + c[K + constrs_len-1:])
dist_stored = np.array(distances)[:dist_gurobi.shape[0]]

Compare the coefficients of variables in the objective from the last iteration of Gurobi with the cluster representations

In [41]:
np.allclose(dist_gurobi,dist_stored)

True

Reoptimize the model using the Gurobi output of the master problem from the last iteration

In [141]:
model.optimize()

Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (win64 - Windows 11+.0 (22631.2))

CPU model: Intel(R) Core(TM) i7-8550U CPU @ 1.80GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 204 rows, 10180 columns and 889443 nonzeros
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [5e-02, 4e+03]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 5e+00]

Solved in 0 iterations and 0.01 seconds (0.00 work units)
Optimal objective  7.764317062e+02


In [144]:
objectives[-1], model.ObjVal

(776.431706195563, 776.431706195563)

In [145]:
last_sol_gurobi = [var.X for var in model.getVars()]
last_sol_gurobi_Z = last_sol_gurobi[:K] + last_sol_gurobi[K + constrs_len-1:]
last_sol_gurobi_slackness = last_sol_gurobi[K:K + constrs_len-1]

In [160]:
np.argmax(last_sol_gurobi_Z - solutions[-1]), np.argmin(last_sol_gurobi_Z - solutions[-1])

(8330, 8319)

In [161]:
np.max(last_sol_gurobi_Z - solutions[-1]), np.min(last_sol_gurobi_Z - solutions[-1])

(0.32663158628942224, -0.32663)

In [162]:
last_sol_gurobi_Z[8330], last_sol_gurobi_Z[8319]

(0.32663158628942224, 0.0)

In [169]:
np.allclose(clusters[8319,:], clusters[8330,:])

True

check partition constraints

In [149]:
partition_constr = [clusters[:solutions[-1].shape[0],i] @ solutions[-1] + slacks[-1][i] for i in range(n)]

In [150]:
np.allclose(np.array(partition_constr),np.ones(n),rtol=1e-3)

True

check number of clusters constraint

In [151]:
np.sum(solutions[-1])

4.99999

checks the cluster representation of t of the same as the t in the LP

In [152]:
m = solutions[-1].shape[0]

In [153]:
t1 = []
t2 = []
for i in (range(m)):
    t_vector = []
    for j in range(l):
        cluster = clusters[i,:n]
        size = sum(cluster)
        group_size = cluster @ q[j]
        if group_size >= alpha * size:
            t_vector.append(1)
        else:
            t_vector.append(0)
    t1.append(t_vector)
    t2.append(clusters[i,n:n+l])

In [154]:
np.allclose(t1,t2,rtol=1e-3)

True

check fairness constraint

In [155]:
for i in range(n,n+l):
    print(solutions[-1] @ clusters[:solutions[-1].shape[0],i] + slacks[-1][i])

0.9999958881474629
1.0
3.7494500000000004


compares the solutions from Gurobi's log file with the stored solutions

In [120]:
import pandas as pd

In [121]:
df = pd.read_csv("./tests/model_write/" + file_name + "_out501.sol", sep = " ")

In [122]:
last_solution_gurobi = df['Solution'][1:].to_numpy(dtype=float)

In [124]:
last_solution_gurobi_Z = np.concatenate((last_solution_gurobi[:K], last_solution_gurobi[K + constrs_len - 1:]))

In [125]:
last_solution_gurobi_slackness = last_solution_gurobi[K:K + constrs_len - 1]

In [131]:
np.allclose(last_solution_gurobi_slackness, slacks[500], atol = 1e-3)

True

In [136]:
np.allclose(last_solution_gurobi_Z, solutions[500][:last_solution_gurobi_Z.shape[0]], atol = 1e-3)

True

Gurobi doesn't generate the same solution for the 2 solves, and it may be caused by similar costs of clusters

## Visualizations

## Parameters

In [23]:
Zvars = model.getVars()[:K] + model.getVars()[K + constrs_len-1:]
Slack_vars_partition = model.getVars()[K : K + n]
Slack_vars_represent = model.getVars()[K + n : K + n + l]

In [24]:
len(Zvars) + len(Slack_vars_partition) + len(Slack_vars_represent) == len(model.getVars())

True

In [25]:
objective = gp.quicksum(Zvars[i] * dist_gurobi[i] for i in range(len(Zvars))) + \
    gp.quicksum(i * 0 for i in Slack_vars_partition) + \
    gp.quicksum(i * 0 for i in Slack_vars_represent)
model.setObjective(objective, GRB.MINIMIZE)
model.optimize()

Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (win64 - Windows 11+.0 (22631.2))

CPU model: Intel(R) Core(TM) i7-8550U CPU @ 1.80GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 204 rows, 10184 columns and 815156 nonzeros
Model fingerprint: 0x2d71cc41
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [2e+00, 4e+03]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 5e+00]


Presolve removed 3 rows and 869 columns
Presolve time: 0.42s
Presolved: 201 rows, 9315 columns, 765410 nonzeros

Concurrent LP optimizer: dual simplex and barrier
Showing barrier log only...

Ordering time: 0.01s

Barrier statistics:
 AA' NZ     : 2.010e+04
 Factor NZ  : 2.030e+04 (roughly 4 MB of memory)
 Factor Ops : 2.727e+06 (less than 1 second per iteration)
 Threads    : 3

Barrier performed 0 iterations in 0.67 seconds (0.34 work units)
Barrier solve interrupted - model solved by another algorithm


Solved with dual simplex
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       6    2.9098429e+01   0.000000e+00   0.000000e+00      1s

Solved in 6 iterations and 0.77 seconds (0.36 work units)
Optimal objective  2.909842886e+01


In [26]:
c2 = [var.obj for var in model.getVars()]

In [27]:
c2 == c

False

In [76]:
def get_nonzeros(l):
    res = []
    for i,j in enumerate(l):
        if j != 0:
            res.append(i)
    #print(l)
    print(res)